<img align="right" src="images/tf-small.png"/>

# BHSA versions

In this notebook we map the slots between the versions `3`, `4`, `4b` and 2016 of the BHSA dataset.
In another notebook,
[mappingNodes](https://github.com/ETCBC/bhsa/blob/master/programs/mappingNodes.ipynb),
we extend this mapping to a mapping between *all* nodes of the versions involved, not just the slots.

The resulting mappings can be used by text-fabric programs to enable themselves to
work on different versions of the data than for which they were developed.

In general, node mappings between versions can not be perfect.

# Slot mappings
We map the slots of a version to the slots of the next version.
Mappings go from old to new, and they are between successive versions.

## Mappings are edges
We store mappings as ordinary TF edge features, so you can use the mapping in both ways, by

```
nodesInVersion4 = Es('omap@3-4').f(nodeInVersion3)
nodesInVersion3 = Es('omap@3-4').t(nodeInVersion4)
```

respectively.

## Differences

When we compare versions, our aim is not to record all differences in general, but to record
the correspondence between the slots of the versions, and exactly where and how this
correspondence is disturbed.

We use the lexeme concept as an anchor point for the correspondence.
If we compare the two versions, slot by slot, and as long as we encounter the same lexemes,
we have an undisturbed correspondence.
In fact, we relax this a little bit, because the very concept of lexeme might change between versions.
So we reduce the information in lexemes considerably, before we compare them, so that we
do not get disturbed by petty changes.

While being undisturbed, we just create an adge between the slot in the one version that we are at,
to the node in the other version that we are at, 
and we assing no value to such an edge.

But eventually, we encounter real disturbances.
They manifest themselves in just a few situations:

1. ![1](diffs/diffs.001.png)
2. ![2](diffs/diffs.002.png)
3. ![3](diffs/diffs.003.png)

In full generality, we can say: $i$ slots in the source version correspond to $j$ slots in the
target version, where $i$ and $j$ may be 0, but not at the same time:

1. ![4](diffs/diffs.004.png)

## Edges carry data
We want to leave behind the information of what happened, and we can do so if we let the edges
carry data.

Suppose we have an edge between node $n$ in the old version and node $m$ in the new version.
Then this is what happens in the general case:

1. If there is no disturbance, we create an edge between $n$ and $m$, with no data associated.
2. If $i$ slots in the source version, starting at $n$ 
   get replaced by $j$ slots in the target version, starting at $m$,
   we create edges between all $n, ..., n+i-1$ on the one hand
   and all $m, ..., m+j-1$ on the other hand,
   and associate them all with the same number $j-i$.

We have a closer look at the edge cases.

### Lexeme change
When a lexeme changes at a particular spot $n, m$, 
we have $i=j=1$, leading to exactly one edge $(n, m)$ with value $0$.

### Slot deletion
When a slot is deleted from the source, we have $i=1, j=0$, leading to 

## Reading an edge
Now suppose we have an old version $V$ and and a new version $W$.
We are processing slots in text in $W$ and want to access information from $V$.
We do so by consulting the mapping feature `omapV-W`.

Say we have arrived at slot $m\in W$ and want to see what is happening at the corresponding place in $V$.

So we retrieve 

```
origins = Es(omapV-W).t(m)
```

Now origins contains the nodes $n$ in $V$ that map to $m$, with additional information.
It is delivered as a dictionary of ($n$, $k$) values, where $k$ is either a number, or `None`.

Let's see what we can deduce, depending on $k$:

1. if we have ($n$, `None`), then the correspondence between $n$ and $m$ is precise.
   Slot $m\inW$ is not the result of a split or collapse in $V$, and both $n$ and $m$ carry the same lexeme.
   There will be no other ($n'$, $k$)s that map to $m$.
1. if we have ($n$, 0), then there is a change of lexeme between $n$ and $m$, but no split or collapse.
   There cannot be other edges arriving at $m$ in this case. For:
   1. If there was also another ($n'$, 0) linked to $m$,
      then $n$ and $n'$ would have been involved in a collapse,
      and the edge value should have been a negative number.
   1. If there was another ($n'$, $k$) with $k$ negative linked to $m$,
      then by the same reasoning the value on the edge between $n$ and $m$ should have been negative.
   1. If there was another ($n'$, $k$) with $k$ positive linked to $m$,
      then $m$ is a part of the result of splitting $n'$.
1. if we have one or more ($n$, $k$)s
   1. $k$ is negative
      in a number of parts. We find those parts
   No splits of collapses.
1. if we have ($n$

# Application: Strong numbers
Stephen Ku has prepared a Strong number mapping for version `4`, based on 
[OpenScriptures Bible Lexicon](https://github.com/openscriptures/HebrewLexicon).

This provides us with a nice use case:
can we apply the Strong number mapping for version `4` to versions `3`, `4b` and `2016`
as well?
See notebook
[strong](https://github.com/ETCBC/bhsa/blob/master/programs/strong.ipynb)
for how we add Strong numbers to the BHSA dataset.

Below we will get a pretty good view on the differences between the versions.
We use the
[BHSA transcription](https://shebanq.ancient-data.org/shebanq/static/docs/BHSA-transcription.pdf)
to write down the diffs.

In [1]:
import os,collections
from utils import caption
from tf.fabric import Fabric

We specify our versions and the subtle differences between them as far as they are relevant.

In [2]:
baseDir = '~/github/etcbc/bhsa/tf'

versions = '''
    3 
    4 
    4b 
    2016
'''.strip().split()

versionInfo = {
    '': dict(
            OCC='g_word',
            LEX='lex',
        ),
    '3': dict(
            OCC='text_plain',
            LEX='lexeme',
        ),
} 

Load all versions in one go!

In [3]:
TF = {}
api = {}
for v in versions:
    for (param, value) in versionInfo.get(v, versionInfo['']).items():
        globals()[param] = value
    caption(4, 'Version -> {} <- loading ...'.format(v))
    TF[v] = Fabric(locations='{}/{}'.format(baseDir, v), modules=[''])
    api[v] = TF[v].load('{} {}'.format(OCC, LEX))

..............................................................................................
.       0.00s Version -> 3 <- loading ...                                                    .
..............................................................................................
This is Text-Fabric 3.0.6
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

118 features found and 0 ignored
  0.00s loading features ...
   |     0.13s B lexeme               from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.18s B text_plain           from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.00s Feature overview: 115 for nodes; 2 for edges; 1 configs; 7 computed
  4.29s All features loaded/computed - for details use loadLog()
..............................................................................................
.       4.

We want to switch easily between the APIs for the versions.

In [4]:
def activate(v):
    for (param, value) in versionInfo.get(v, versionInfo['']).items():
        globals()[param] = value
    api[v].makeAvailableIn(globals())
    caption(4, 'Active version is now -> {} <-'.format(v))

Inspect the amount of slots in all versions.

In [5]:
nSlots = {}
for v in versions:
    activate(v)
    nSlots[v] = F.otype.maxSlot
    caption(0, '\t {} slots'.format(nSlots[v]))

..............................................................................................
.         28s Active version is now -> 3 <-                                                  .
..............................................................................................
|         28s 	 426499 slots
..............................................................................................
.         28s Active version is now -> 4 <-                                                  .
..............................................................................................
|         28s 	 426555 slots
..............................................................................................
.         28s Active version is now -> 4b <-                                                 .
..............................................................................................
|         28s 	 426568 slots
..........................................................

# Method

When we compare two versions, we inspect the lexemes found at corresponding positions in the versions.
We start at the beginning, and when the lexemes do not match, we have a closer look.

However, in order not to be disturbed by minor discrepancies in the lexemes, we mask the lexemes: we
apply a few transformations to it, such as removing alefs and waws, and finally even turning them into
ordered sets of letters, thereby loosing the order and multiplicity of letter.
We also strip the disambiguation marks.

We maintain a current mapping between the slots of the two versions, and we update it if we encounter
disturbances. 
Initially, this map is the identity map.

What we encounter as remaining differences boils down to the following:

* a lexeme is split into two lexemes with the same total material, typically involving `H`, `MN`, or `B`
* the lexeme is part of a special case, listed in the `cases` table (which has been found by repeatedly
  chasing for the first remaining difference.
* the both lexemes differ, but that's it, no map updates have to be done.
  
The first two types of cases can be solved by splitting a lexeme into `k` parts or combining `k` lexemes into one.
After that the mapping has to be shifted to the right or to the left from a certain point onwards.

The loop then is as follows:

* find the first slot with a lexeme in the first version that is different from the lexeme at the mapped slot
  in the second version
* analyse what is the case:
  * if the disturbance is recognized on the basis of existing patterns and cases, update the map and
    consider this case solved
  * if the disturbance is not recognized, the case is unsolved, and we break out of the loop.
    More analysis is needed, and the outcome of that has to be coded as an extra pattern or case.
* if the status is solved, go back to the first step

We end up with a mapping from the slots of the first version to those of the other version that links
slots with approximately equal lexemes together.

# Lexeme masking
We start by defining our masking function, and compile lists of all lexemes and masked lexemes for all versions.

In [6]:
masks = [
    (lambda lex: lex.rstrip('[/='),                         'strip disambiguation'),
    (lambda lex: lex[0:-2] if lex.endswith('JM') else lex,  'remove JM'),
    (lambda lex: lex[0:-2] if lex.endswith('WT') else lex,  'remove WT'),
    (lambda lex: lex.replace('J', ''),                      'remove J'),
    (lambda lex: lex.replace('>', ''),                      'remove Alef'),
    (lambda lex: lex.replace('W', ''),                      'remove W'),
    (lambda lex: lex.replace('Z', 'N'),                     'identify Z and N'),
    (lambda lex: lex.rstrip('HT'),                          'strip HT'),
    (lambda lex: (''.join(sorted(set(set(lex)))))+'_'*lex.count('_'), 'ignore order and multiplicity'),
]

def mask(lex, trans=None):
    if trans != None:
        return masks[trans][0](lex)
    for (fun, desc) in masks:
        lex = fun(lex)
    return lex

Carry out the lexeme masking for all versions.

In [7]:
lexemes = {}

caption(4, 'Masking lexemes')
for v in versions:
    activate(v)
    lexemes[v] = collections.OrderedDict()
    for n in F.otype.s('word'):
        lex = Fs(LEX).v(n)
        lexemes[v][n] = (lex, mask(lex, trans=0), mask(lex))
caption(0, 'Done')

..............................................................................................
.         35s Masking lexemes                                                                .
..............................................................................................
..............................................................................................
.         35s Active version is now -> 3 <-                                                  .
..............................................................................................
..............................................................................................
.         38s Active version is now -> 4 <-                                                  .
..............................................................................................
..............................................................................................
.         41s Active version is now -> 4b <-      

# Cases and mappings
In `cases` we store special cases that we stumbled upon.
Every time we encountered a disturbance which did not follow a recognized pattern,
we turned it into a case.
The number is the slot number in the first version where the case will be applied.
Cases will only be applied at these exact slot number and nowhere else.

In [8]:
cases = {}
mappings = {}

# Algorithm

Here is the code that directly implements the method.
Every pair of distinct versions can be mapped.
We store the mappings in a dictionary, keyed by tuples like `(4, 4b)`, 
for the mapping from version `4` to `4b`, for instance.

The loop is in `doDiffs` below.

In [9]:
def inspect(v1, v2, start, end):
    mapKey = (v1, v2)
    mp = mappings[mapKey]
    for n in range(start, end):
        print('{:>6}: {:<8} {:<8}'.format(
            n, 
            api[v1].Fs(LEX).v(n),
            api[v2].Fs(LEX).v(mp[n]),

        ))

def firstDiff(v1, v2, start):
    mapKey = (v1, v2)
    mp = mappings[mapKey]

    fDiff = None
    for (n, (lx1, sxl, mx1)) in lexemes[v1].items():
        if n < start: continue
        if mx1 != lexemes[v2][mp[n]][2]:
            fDiff = n
            break
    return fDiff

def printDiff(v1, v2, n):
    mapKey = (v1, v2)
    mp = mappings[mapKey]

    (lx1, sx1, mx1) = lexemes[v1][n]
    (lx2, sx2, mx2) = lexemes[v2][mp[n]]
    if n < api[v1].F.otype.maxSlot:
        (lx1n, sx1n, mx1n) = lexemes[v1][n+1]
    else:
        (lx1n, sx1n, mx1n) = ('max', 'max', 'max')
    if mp[n] < api[v2].F.otype.maxSlot:
        (lx2n, sx2n, mx2n) = lexemes[v2][mp[n+1]]
    else:
        (lx2n, sx2n, mx2n) = ('max', 'max', 'max')
    if n > 1:
        (lx1p, sx1p, mx1p) = lexemes[v1][n-1]
    else:
        (lx1p, sx1p, mx1p) = ('min', 'min', 'min')
    if mp[n] > 1:
        (lx2p, sx2p, mx2p) = lexemes[v2][mp[n-1]]
    else:
        (lx2p, sx2p, mx2p) = ('min', 'min', 'min')

    #print('''{} {}:{} ==> slot {} ==> {}
    #{:<2}: {:<6} ~ |{:<6}| ~ {:<6}   {:<6} ~ |{:<6}| ~ {:<6}   {:<6} ~ |{:<6}| {:<6}
    #{:<2}: {:<6} ~ |{:<6}| ~ {:<6}   {:<6} ~ |{:<6}| ~ {:<6}   {:<6} ~ |{:<6}| {:<6}'''.format(
    #    *api[v1].T.sectionFromNode(n),
    #    n, mp[n],
    #    v1, lx1p, lx1, lx1n, sx1p, sx1, sx1n, mx1p, mx1, mx1n,
    #    v2, lx2p, lx2, lx2n, sx2p, sx2, sx2n, mx2p, mx2, mx2n,
    #)) 
    print('''{} {}:{} ==> slot {} ==> {}
    {:>4}: ┣{:<6}┫ ▷{:>10}◁▶{:>10}◀▷{:<8}◁
    {:>4}: ┣{:<6}┫ ▷{:>10}◁▶{:>10}◀▷{:<8}◁'''.format(
        *api[v1].T.sectionFromNode(n),
        n, mp[n],
        v1, mx1, lx1p, lx1, lx1n, 
        v2, mx2, lx2p, lx2, lx2n,
    )) 

# doDiffs

This function contains the loop to walk through all differences.

In [10]:
MAX_ITER = 250

def doDiffs(v1, v2):
    mapKey = (v1, v2)
    mappings[mapKey] = dict(((n, n) for n in api[v1].F.otype.s('word')))
    mp = mappings[mapKey]
    theseCases = cases.get(mapKey, {})
    it = 0
    start = 1
    while True:
        n = firstDiff(v1, v2, start)

        if n == None:
            print('No more differences.\nFound {} points of disturbance'.format(it))
            break

        if it > MAX_ITER: 
            print('There might be more disturbances: increase MAX_ITER')
            break
            
        it += 1

        printDiff(v1, v2, n)

        (lx1, sx1, mx1) = lexemes[v1][n]
        (lx2, sx2, mx2) = lexemes[v2][mp[n]]
        (lx1n, sx1n, mx1n) = lexemes[v1][n+1]
        (lx2n, sx2n, mx2n) = lexemes[v2][mp[n+1]]

        solved = None
        skip = 0
        if n in theseCases:
            (action, param) = theseCases[n]
            if action == 'collapse':
                solved = '{} {} slots'.format(action, param)
                skip = param
                for m in range(api[v1].F.otype.maxSlot, n + param -1, -1):
                    mp[m] = mp[m-param+1]
                for m in range(n+1, n+param):
                    mp[m] = mp[n]
            elif action == 'split':
                solved = '{} into {} slots'.format(action, param)
                for m in range(n+1, api[v1].F.otype.maxSlot+1):
                    mp[m] = mp[m] + param -1
            elif action == 'ok':
                solved = 'incidental variation in lexeme'
#        elif lx1.replace('C', 'X') == lx2:
#            solved = 'letter C replaced by X'
        elif lx1 in theseCases:
            (action, param) = theseCases[lx1]
            if action == 'ok':            
                if lx2 == param:
                    solved = 'systematic variation in lexeme' 
            elif action == 'split':
                solved = 'systematic {} on _ into {} slots'.format(action, param)
                for m in range(n+1, api[v1].F.otype.maxSlot+1):
                    mp[m] = mp[m] + param -1
        elif '_' in lx1:
            action = 'split'
            param = lx1.count('_') + 1
            solved = '{} on _ into {} slots'.format(action, param)
            for m in range(n+1, api[v1].F.otype.maxSlot+1):
                mp[m] = mp[m] + param -1
        elif lx1 == lx2 + lx2n:
            if lx2 == 'H':
                solved = 'split article off'
                for m in range(n+1, api[v1].F.otype.maxSlot+1):
                    mp[m] = mp[m] + 1
        elif set(mx1) == set(mx2) | set(mx2n):
            if lx2 == 'B' or lx2 == 'MN':
                solved = 'split preposition off'
                for m in range(n+1, api[v1].F.otype.maxSlot+1):
                    mp[m] = mp[m] + 1
        print('Action: {}\n'.format(solved if solved else 'BLOCKED'))

        if not solved: break
        
        start = n + 1 + skip

    if not solved:
        print('Blocking difference in {} iterations'.format(it))

The mappings itself are needed elsewhere in Text-Fabric, let us write them to file.
We write them into the dataset corresponding to the target version.
So the map `3-4` ends up in version `4`.

In [11]:
def writeMaps():
    for ((v1, v2), mp) in mappings.items():
        fName = 'omap@{}-{}'.format(v1, v2)
        caption(4, 'Write slot mapping {}'.format(fName))

        edgeFeatures = {
            fName: dict(((n, (mp[n],)) for n in range(1, api[v1].F.otype.maxSlot + 1)))
        }
        metaData = {
            fName: {
                'about': 'Mapping from the slots of BHSA version {} to version {}'.format(v1, v2),
                'encoder': 'Dirk Roorda by a semi-automatic method',
                'see': 'https://github.com/ETCBC/bhsa/blob/master/programs/evolutionVersions.ipynb',
                'valueType': 'str',
            }
        }
        TF[v2].save(
            nodeFeatures={},
            edgeFeatures=edgeFeatures,
            metaData=metaData,
        )

# Running

Here we run the mapping between `3` and `4`.

## 3 => 4

Here are the special cases for this conversion.

In [16]:
cases[('3', '4')] = {}

In [17]:
cases.update({
    ('3', '4'): {
#        7840: ('ok', None),
        'CXH[' : ('ok', 'XWH['),
    },
})

In [14]:
cases.update({
    ('3', '4'): {
        'CXH[' : ('ok', 'XWH['),
        'MQYT/': ('split', 2),
        28730  : ('ok', None),
        121812 : ('ok', None),
        174515 : ('ok', None),
        201089 : ('ok', None),
        218383 : ('split', 3),
        221436 : ('ok', None),
        247730 : ('ok', None),
        272884 : ('collapse', 2),
        353611 : ('ok', None),
#        370037 : ('split', 2),
#        370138 : ('split', 2),
#        370329 : ('split', 2),
    },
})    

In [18]:
doDiffs('3', '4')

Genesis 18:2 ==> slot 7840 ==> 7840
       3: ┣CX    ┫ ▷         W◁▶      CXH[◀▷>RY/    ◁
       4: ┣X     ┫ ▷         W◁▶      XWH[◀▷>RY/    ◁
Action: systematic variation in lexeme

Genesis 19:1 ==> slot 8447 ==> 8447
       3: ┣CX    ┫ ▷         W◁▶      CXH[◀▷>P/     ◁
       4: ┣X     ┫ ▷         W◁▶      XWH[◀▷>P/     ◁
Action: systematic variation in lexeme

Genesis 21:14 ==> slot 9856 ==> 9856
       3: ┣<BCR__┫ ▷     MDBR/◁▶  B>R_CB</◀▷W       ◁
       4: ┣BR    ┫ ▷     MDBR/◁▶      B>R/◀▷CB<==/  ◁
Action: split on _ into 2 slots

Genesis 21:31 ==> slot 10174 ==> 10175
       3: ┣<BCR__┫ ▷       HW>◁▶  B>R_CB</◀▷KJ      ◁
       4: ┣BR    ┫ ▷       HW>◁▶      B>R/◀▷CB<==/  ◁
Action: split on _ into 2 slots

Genesis 21:32 ==> slot 10183 ==> 10185
       3: ┣<BCR__┫ ▷         B◁▶  B>R_CB</◀▷W       ◁
       4: ┣BR    ┫ ▷         B◁▶      B>R/◀▷CB<==/  ◁
Action: split on _ into 2 slots

Genesis 21:33 ==> slot 10200 ==> 10203
       3: ┣<BCR__┫ ▷         B◁▶  B>R_CB</◀▷W       ◁
 

# Running

Here we run the mapping between `4` and `4b`.
The points of disturbance will be written into the output cell.

## 4 => 4b

Here are the special cases for this conversion.

In [16]:
cases.update({
    ('4', '4b'): {
        214730: ('collapse', 4),
        260028: ('split', 2),
        289948: ('ok', None),
        307578: ('split', 2),
        323067: ('ok', None),
        389774: ('ok', None),
        407543: ('split', 2),
        408429: ('split', 2),
    },
})

In [17]:
doDiffs('4', '4b')

Genesis 24:65 ==> slot 12369 ==> 12369
       4: ┣HLN   ┫ ▷      >JC/◁▶      HLZH◀▷H       ◁
      4b: ┣      ┫ ▷      >JC/◁▶         H◀▷LZH     ◁
Action: split article off

Genesis 37:19 ==> slot 20514 ==> 20515
       4: ┣HLN   ┫ ▷     XLWM/◁▶      HLZH◀▷BW>[    ◁
      4b: ┣      ┫ ▷     XLWM/◁▶         H◀▷LZH     ◁
Action: split article off

Judges 6:20 ==> slot 130846 ==> 130848
       4: ┣HLN   ┫ ▷      SL</◁▶       HLZ◀▷W       ◁
      4b: ┣      ┫ ▷      SL</◁▶         H◀▷LZ      ◁
Action: split article off

1_Samuel 14:1 ==> slot 148319 ==> 148322
       4: ┣HLN   ┫ ▷      <BR/◁▶       HLZ◀▷W       ◁
      4b: ┣      ┫ ▷      <BR/◁▶         H◀▷LZ      ◁
Action: split article off

1_Samuel 17:26 ==> slot 151331 ==> 151335
       4: ┣HLN   ┫ ▷    PLCTJ/◁▶       HLZ◀▷W       ◁
      4b: ┣      ┫ ▷    PLCTJ/◁▶         H◀▷LZ      ◁
Action: split article off

1_Samuel 20:19 ==> slot 153816 ==> 153821
       4: ┣HLN   ┫ ▷      >BN/◁▶     H>ZL/◀▷W       ◁
      4b: ┣      ┫ ▷      >BN

Just have a look at the first point of disturbance:

In [18]:
(v1, v2) = ('4', '4b')
(n, m) = [x for x in mappings[(v1, v2)].items() if x[0] != x[1]][0]
print('{} {}:{} node {}: {} versus {} becomes {}'.format(
    *api[v1].T.sectionFromNode(n),
    n,
    api[v1].F.lex.v(n),
    api[v2].F.lex.v(n),
    api[v2].F.lex.v(m),
))

Genesis 24:65 node 12370: H versus LZH becomes H


## 4b => 2016

We need other cases.

In [19]:
cases.update({
    ('4b', '2016'): {
         28423: ('split', 3),
         28455: ('split', 3),
         91193: ('split', 2),
         91197: ('split', 2),
        122218: ('split', 2),
        122247: ('split', 2),
        123160: ('split', 2),
        184086: ('split', 2),
        394186: ('collapse', 2),
        395150: ('ok', None),
        395190: ('ok', None),
        401036: ('split', 3),
        404503: ('ok', None),
        419138: ('split', 3),
    },    
})

In [20]:
doDiffs('4b', '2016')

Genesis 50:10 ==> slot 28423 ==> 28423
      4b: ┣DGNRV__┫ ▷        <D◁▶  GRN_>VD/◀▷>CR     ◁
    2016: ┣GNR   ┫ ▷        <D◁▶      GRN/◀▷H       ◁
Action: split into 3 slots

Genesis 50:11 ==> slot 28455 ==> 28457
      4b: ┣DGNRV__┫ ▷         B◁▶  GRN_>VD/◀▷W       ◁
    2016: ┣GNR   ┫ ▷         B◁▶      GRN/◀▷H       ◁
Action: split into 3 slots

Numbers 33:45 ==> slot 91193 ==> 91197
      4b: ┣BDGN__┫ ▷         B◁▶ DJBWN_GD/◀▷W       ◁
    2016: ┣BDN   ┫ ▷         B◁▶     DJBN/◀▷GD==/   ◁
Action: split into 2 slots

Numbers 33:46 ==> slot 91197 ==> 91202
      4b: ┣BDGN__┫ ▷        MN◁▶ DJBWN_GD/◀▷W       ◁
    2016: ┣BDN   ┫ ▷        MN◁▶     DJBN/◀▷GD==/   ◁
Action: split into 2 slots

Joshua 16:3 ==> slot 122218 ==> 122224
      4b: ┣BNRTX___┫ ▷     GBWL/◁▶BJT_XRWN_TXTWN/◀▷W       ◁
    2016: ┣BNRTX__┫ ▷     GBWL/◁▶BJT_XWRWN/◀▷TXTWN/  ◁
Action: split into 2 slots

Joshua 16:5 ==> slot 122247 ==> 122254
      4b: ┣<BLNRTX___┫ ▷        <D◁▶BJT_XRWN_<LJWN/◀▷W       ◁
    2016: ┣BN

The bit below is very handy if you need a closer look to what is the case in some range of slots.

In [21]:
inspect('4b', '2016', 419135, 419145)

419135: <M/      <M/     
419136: W        W       
419137: HLK[     HLK[    
419138: GJ>_MLX/ GJ>/    
419139: W        W       
419140: NKH[     NKH[    
419141: >T       >T      
419142: BN/      BN/     
419143: F<JR====/ F<JR====/
419144: <FRH=/   <FRH=/  


Just have a look at the first point of disturbance:

In [22]:
(v1, v2) = ('4b', '2016')
(n, m) = [x for x in mappings[(v1, v2)].items() if x[0] != x[1]][0]
print('{} {}:{} node {}: {} versus {} becomes {}'.format(
    *api[v1].T.sectionFromNode(n),
    n,
    api[v1].F.lex.v(n),
    api[v2].F.lex.v(n),
    api[v2].F.lex.v(m),
))

Genesis 50:10 node 28424: >CR versus H becomes >CR


In [26]:
caption(4, 'Constructed mappings:')
for (v1, v2) in sorted(mappings.keys()):
    caption(0, '\t {:>4} ==> {:<4}'.format(v1, v2))

writeMaps()

..............................................................................................
.      7m 19s Constructed mappings:                                                          .
..............................................................................................
|      7m 19s 	    3 ==> 4   
|      7m 19s 	    4 ==> 4b  
|      7m 19s 	   4b ==> 2016
..............................................................................................
.      7m 19s Write slot mapping omap@3-4                                                    .
..............................................................................................
  0.00s Exporting 0 node and 1 edge and 0 config features to /Users/dirk/github/etcbc/bhsa/tf/4:
   |     1.26s T omap@3-4             to /Users/dirk/github/etcbc/bhsa/tf/4
  1.26s Exported 0 node features and 1 edge features and 0 config features to /Users/dirk/github/etcbc/bhsa/tf/4
......................................................